In [1]:
!pip install ensemble_boxes

     |████████████████████████████████| 3.3 MB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 76.3 MB/s eta 0:00:01    |█▉                              | 1.9 MB 76.3 MB/s eta 0:00:01     |███▉                            | 4.2 MB 76.3 MB/s eta 0:00:01


In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# ensemble csv files
submission_files = ['./cascade_rcnn.csv',
                   './atss_dyhead.csv'] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

ensemble_submission = './atss_dyhead+cascade_rcnn_0.05.csv'

In [3]:
image_ids = submission_df[0]['image_id'].tolist()

In [5]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.6

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [7]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(ensemble_submission)

submission.head()

,PredictionString,image_id
0,7 0.9127387404441833 606.1720581054688 507.687...,test/0000.jpg
1,5 0.914823591709137 137.24798583984375 1.04625...,test/0001.jpg
2,1 0.6799492239952087 291.4575500488281 300.297...,test/0002.jpg
3,9 0.5813835859298706 120.2798080444336 236.532...,test/0003.jpg
4,0 0.5973770022392273 450.7486267089844 406.322...,test/0004.jpg
